# Merge DataFrame
- merge clear sky irradiance, measurement data and cloud information (all_ci_df.csv) together.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from  datetime import timedelta

import warnings
warnings.filterwarnings("ignore")

## ML

In [ ]:
iclr_df = pd.read_csv('iclr56sites_Jan2022_Feb2023.csv')
cloud_df = pd.read_csv('all_ci_df1e0.csv')

In [ ]:
# merge all DataFrame

all_sites_no = 56
site_no_list = [i for i in range(1, all_sites_no + 1)]

# Step 0 create null DataFrame to store upcomming data
all_sites_df = pd.DataFrame()

# loop over site  

for site_no in site_no_list:
    
    print(f'Stacking site {site_no}')
        
    # Step 1 read measurement data from file name then assign columns name and site number column 
    
    file_name = f'site{("00"+str(site_no))[-3:]}.csv'
    site_df =  pd.read_csv(file_name, parse_dates=['Datetime'], skiprows=1,names=['Datetime', 'P', 
                                                                                  'T_ambient','I', 'T_pv'])
    if site_no == 54:
        print(f'pass site {site_no}')
        continue
    elif site_no == 32:
        site_df = site_df.set_index('Datetime')
        site_df = site_df.loc[:'2022-05-01']
        print('filter on to May 2022 , site = 32')
        site_df = site_df.reset_index()
        
    mask = (site_df['P'] > 10) & (site_df['I'] ==0)
    site_df = site_df[~mask]
#     site_df = site_df.set_index('Datetime')
    
#     if site_df.empty:
#         continue
    
    
#     site_df = site_df.resample('30min').asfreq()
#     site_df = site_df.between_time('06:00:00', '17:00:00')
#     dt = timedelta(days = 7)
    
#     for col in site_df.columns :
#         missing_datetime_list = site_df[site_df[col].isnull()].index
#         for missing_datetime in missing_datetime_list:
#             missing_datetime_start = missing_datetime-dt
#             missing_datetime_end = missing_datetime+dt
            
#             t = missing_datetime.time()
#             slice_df = site_df.loc[missing_datetime_start:missing_datetime_end]
#             site_df.loc[missing_datetime] = slice_df[slice_df.index.time == t].mean() 
    
#     site_df = site_df.reset_index()
    
    # Step 2 choose cloud information only selected site 
    
    cloud_site_df = cloud_df[cloud_df['site']==site_no].iloc[:,1:]
    cloud_site_df['Datetime'] = cloud_site_df['Datetime'].astype('datetime64[ns]')
    
    
    # Step 3 choose clear sky irradiance 
    
    iclr_site_df = iclr_df[['Datetime', f'iclr_{site_no_str}', 'hour_index']]
    iclr_site_df.rename(columns={f'iclr_{site_no_str}' : 'iclr'}, inplace=True)
    iclr_site_df['Datetime'] = iclr_site_df['Datetime'].astype('datetime64[ns]')
    
    # Step 4 merge measurement ,cloud information DataFrame
    
    site_df = pd.merge(site_df, cloud_site_df, on='Datetime', how='inner')
        
    site_df = pd.merge(site_df, iclr_site_df , on='Datetime', how='inner')
    site_df['Datetime'] = pd.to_datetime(site_df['Datetime'])
    site_df = site_df.set_index('Datetime')
    
    # Step 5 create feature 
    
    # clear sky index k
    site_df['k'] = site_df['I'] / site_df['iclr']
#     mask = site_df['k'] >= 1.3
    
#     site_df = site_df[~mask]
#     site_df['I_cal'] = site_df['iclr'] * site_df['ci_center'] 
    
    # lag features
    for col in ['P','T_ambient', 'I', 'T_pv'] :
        for i in range(1,7) :
            site_df[f'{col}_lag_{i}step'] = site_df[col].shift(freq=f'{30*i}min')
            
    # lead features
    for col in site_df[['T_ambient', 'I', 'T_pv', 'iclr', 'ci_center', 'k', 'P', 'hour_index']].columns :
        for i in range(1,9) :
            site_df[f'{col}_lead_{i}step'] = site_df[col].shift(-30*i, freq='min')
#             site_df[f'I_cmv_lead_{i}step'] = site_df['iclr'].shift(-30*i, freq='min') * (1-site_df[f'ci_est(t+{i})'])
    # ema irradiance
    alpha = 0.8
    
    site_df['I_ema_t+1'] = alpha * site_df['I'] + (1-alpha)*(site_df['I_lag_1step']) 
    
    # I previous day 
    for i in range(1,9):
        site_df[f'I_lead_{i}step_back1D'] = site_df[f'I_lead_{i}step'].shift(freq='1D')
        
    site_df = site_df.reset_index()
    
    # Step 6 stack over 56 sites
    
    all_sites_df = pd.concat([all_sites_df, site_df], ignore_index=True)


In [ ]:
all_sites_df.to_csv('processed_all_sites_HS1e_0_df_not_imputed_R_channel.csv')

## CNN

In [ ]:
label_df = pd.read_csv('label_df_r.csv')
label_df = label_df.iloc[1:, 1:]
label_df['filename'] = label_df['filename'].astype(str)
label_df['site'] = label_df['filename'].str[-19:-17].astype('int')

label_df['year'] = label_df['filename'].str[-16:-12]
label_df['month'] = label_df['filename'].str[-12:-10]
label_df['day'] = label_df['filename'].str[-10:-8]
label_df['hour'] = label_df['filename'].str[-8:-6]
label_df['minute'] = label_df['filename'].str[-6:-4]

label_df['Datetime'] = label_df['year'] + '-' + label_df['month'] + '-' + label_df['day'] + ' ' + label_df['hour'] + ':' + label_df['minute'] + ':00'        
label_df['Datetime'] = pd.to_datetime(label_df['Datetime'])

label_df = label_df.drop(columns = ['month', 'year', 'day', 'hour', 'minute'])
label_df = label_df.sort_values('site')
label_df

In [ ]:
# merge measurement and iclr
all_sites_df = pd.DataFrame()
all_sites_no = 56
site_no_list = [i for i in range(1, all_sites_no + 1)]

# Step 0 create null DataFrame to store upcomming data
for site_no in site_no_list:
    
    print(f'Stacking site {site_no}')
    
    # Step 1 read measurement data from file name then assign columns name and site number column 
    
    file_name = f'site{("00"+str(site_no))[-3:]}.csv'
    site_df =  pd.read_csv(file_name, parse_dates=['Datetime'], skiprows=1,names=['Datetime', 'P', 
                                                                                  'T_ambient','I', 'T_pv'])
    
    if site_no == 54:
        print(f'pass site {site_no}')
        continue
    elif site_no == 32:
        site_df = site_df.set_index('Datetime')
        site_df = site_df.loc[:'2022-05-01']
        print('filter on to May 2022 , site = 32')
        site_df = site_df.reset_index()
        
    mask = (site_df['P'] > 10) & (site_df['I'] ==0)
    site_df = site_df[~mask]
    
    # Step 2 choose cloud information only selected site 
    
    cloud_site_df = label_df[label_df['site']==site_no]
    cloud_site_df['Datetime'] = cloud_site_df['Datetime'].astype('datetime64[ns]')
    
    # Step 3 choose clear sky irradiance 
    iclr_site_df = iclr_df[['Datetime', f'iclr_{site_no_str}', 'hour_index']]
    iclr_site_df.rename(columns={f'iclr_{site_no_str}' : 'iclr'}, inplace=True)
    iclr_site_df['Datetime'] = iclr_site_df['Datetime'].astype('datetime64[ns]')
    
    # Step 4 merge measurement ,cloud information DataFrame
    
    site_df = pd.merge(site_df, cloud_site_df, on='Datetime', how='inner')
        
    site_df = pd.merge(site_df, iclr_site_df , on='Datetime', how='inner')
    site_df['Datetime'] = pd.to_datetime(site_df['Datetime'])
    site_df = site_df.set_index('Datetime')
    
    # Step 5 create feature 
    
    # clear sky index k
    site_df['k'] = site_df['I'] / site_df['iclr']
#     site_df['I_cal'] = site_df['iclr'] * site_df['ci_center'] 
    
    # lag features
    for col in ['P','T_ambient', 'I', 'T_pv'] :
        for i in range(1,7) :
            site_df[f'{col}_lag_{i}step'] = site_df[col].shift(freq=f'{30*i}min')
            
    # lead features
    for col in site_df[['T_ambient', 'I', 'T_pv', 'iclr','k', 'P', 'hour_index']].columns :
        for i in range(1,9) :
            site_df[f'{col}_lead_{i}step'] = site_df[col].shift(-30*i, freq='min')
#             site_df[f'I_cmv_lead_{i}step'] = site_df['iclr'].shift(-30*i, freq='min') * (1-site_df[f'ci_est(t+{i})'])
    # ema irradiance
    alpha = 0.8
    
    site_df['I_ema_t+1'] = alpha * site_df['I'] + (1-alpha)*(site_df['I_lag_1step']) 
    
    # I previous day 
    for i in range(1,9):
        site_df[f'I_lead_{i}step_back1D'] = site_df[f'I_lead_{i}step'].shift(freq='1D')
        
    site_df = site_df.reset_index()
    
    # Step 6 stack over 56 sites
    
    all_sites_df = pd.concat([all_sites_df, site_df], ignore_index=True)

In [ ]:
all_sites_df.to_csv('df_cnn_r.csv')